In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing as pp

In [2]:
import tensorflow as tf
import keras
from keras import regularizers
from keras.layers import Dense, Activation, Dropout
from keras.layers import BatchNormalization, Input, Lambda
from keras.models import Model, Sequential
from keras.models import load_model

2025-06-23 12:56:13.238467: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
        tf.config.set_soft_device_placement(True)
        growth = tf.config.experimental.get_memory_growth(device)
        print('{} memory growth: {}'.format(device, growth))
else:
    print("Not enough GPU hardware devices available.")

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') memory growth: True
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU') memory growth: True


2025-06-23 12:56:15.372246: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2025-06-23 12:56:15.373745: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2025-06-23 12:56:15.436911: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-06-23 12:56:15.437738: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2025-06-23 12:56:15.437823: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-0

In [4]:
def corre_plot_AE(AE_df_train, X_train_pred):
    x = np.linspace(-2,10)
    y = x
    plt.figure(constrained_layout=True)
    plt.title('train correlation')
    plt.scatter(AE_df_train, X_train_pred, alpha=0.02, s=1)
    plt.plot(x,y,color='green', linestyle = '--')

In [5]:
def correlation_coefficient(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    numerator = np.sum((x - x_mean) * (y - y_mean))
    denominator = np.sqrt(np.sum((x - x_mean) ** 2) * np.sum((y - y_mean) ** 2))
    return numerator / denominator

In [6]:
df_latent_20dims = pd.read_csv('re-latent_df_20dims_PD_HD.csv')
df_latent_20dims

,column 1,column 2,column 3,column 4,column 5,column 6,column 7,column 8,column 9,column 10,...,column 12,column 13,column 14,column 15,column 16,column 17,column 18,column 19,column 20,patient number
0,-4.894204,4.146495,-10.627996,4.333094,-0.211089,-6.648519,-4.280519,-3.815852,-3.777365,-1.499009,...,-7.137741,-5.564987,1.921789,-6.913729,7.227826,-7.480397,-3.841025,-2.643135,-4.290059,1
1,-2.944945,-4.172675,2.317636,3.224382,2.160191,3.711342,5.053267,-7.703062,8.844686,-1.244160,...,5.201087,-5.925610,4.695848,3.283216,-3.963865,5.261399,-4.708281,2.653075,-5.063646,0
2,5.588860,-0.694617,2.670450,-8.996168,-3.310686,-7.067754,3.733333,10.910929,2.443390,-4.109570,...,6.122800,6.156366,-1.021480,3.269602,2.010873,1.207475,5.214314,6.464255,-2.690952,2
3,-7.072881,-4.657294,-6.835857,9.598818,1.465891,-5.823493,-5.777774,-7.312018,-0.630546,-6.401748,...,3.504016,4.543265,6.947121,8.620768,0.387783,-1.047444,3.723100,5.218976,4.335200,1
4,-5.568182,9.380668,7.979990,1.130456,5.788285,-3.061255,-3.599283,6.892598,2.539938,2.496101,...,8.950968,-4.972475,-5.133507,-7.145875,-4.389797,6.115207,-6.742897,5.260499,-5.494330,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,3.641945,-7.661404,7.613710,-1.599616,7.090221,6.726690,-2.069273,-2.745076,2.530561,2.660381,...,8.682987,6.308665,5.456786,8.533322,-3.954587,6.808384,-5.964357,-6.926980,-4.553753,0
104,3.359315,-4.861623,-5.498817,6.374908,1.541136,-6.441969,6.099424,5.147042,-2.580868,-4.977870,...,-3.195096,5.148294,-4.076187,-7.195958,6.914922,-6.155535,4.881458,-7.642689,-8.291637,2
105,-0.756402,0.441134,-6.646461,1.472855,-6.947612,7.541595,4.625729,-6.928267,4.398931,1.071311,...,-5.603864,4.042096,-0.611265,-1.356958,-5.764072,-6.192120,-8.153080,-6.249497,5.730522,1
106,0.938262,-3.290774,6.039089,-9.971478,4.504210,-7.470419,-7.287096,-3.256257,-2.944622,1.461293,...,5.424419,-4.540638,-0.372598,-7.300921,-0.119995,-5.317137,-6.542215,-6.035981,4.083077,1


In [7]:
df_latent_train_20dims = pd.read_csv('re-latent_df_train_20dims_PD_HD.csv')
df_latent_train_20dims

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,patient number
0,-4.894204,4.146495,-10.627996,4.333094,-0.211089,-6.648519,-4.280519,-3.815852,-3.777365,-1.499009,...,-7.137741,-5.564987,1.921789,-6.913729,7.227826,-7.480397,-3.841025,-2.643135,-4.290059,1
1,-2.944945,-4.172675,2.317636,3.224382,2.160191,3.711342,5.053267,-7.703062,8.844686,-1.244160,...,5.201087,-5.925610,4.695848,3.283216,-3.963865,5.261399,-4.708281,2.653075,-5.063646,0
2,5.588860,-0.694617,2.670450,-8.996168,-3.310686,-7.067754,3.733333,10.910929,2.443390,-4.109570,...,6.122800,6.156366,-1.021480,3.269602,2.010873,1.207475,5.214314,6.464255,-2.690952,2
3,-7.072881,-4.657294,-6.835857,9.598818,1.465891,-5.823493,-5.777774,-7.312018,-0.630546,-6.401748,...,3.504016,4.543265,6.947121,8.620768,0.387783,-1.047444,3.723100,5.218976,4.335200,1
4,-5.568182,9.380668,7.979990,1.130456,5.788285,-3.061255,-3.599283,6.892598,2.539938,2.496101,...,8.950968,-4.972475,-5.133507,-7.145875,-4.389797,6.115207,-6.742897,5.260499,-5.494330,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,3.891827,2.561099,6.970787,-7.018413,-5.253020,3.758243,3.671204,-10.044440,-7.940752,-0.323961,...,-5.004793,1.852236,5.476236,-3.787934,-4.054970,2.748752,6.475208,0.135382,-7.873879,2
98,-7.752233,-4.231459,8.498736,-5.204750,-0.988339,2.461777,-1.641066,-7.762600,-0.770963,6.920038,...,-3.921720,-4.659022,-4.321660,2.040658,-4.948811,7.810775,-6.515944,4.377778,4.249447,0
99,5.738795,3.508221,-3.084219,-2.055181,-6.202135,-2.324691,-3.598113,-4.882000,4.343721,3.288866,...,-8.157871,-1.671575,-0.765159,-4.126219,5.952621,-7.953374,-6.093229,4.663666,4.219165,1
100,-1.368667,-3.972085,5.433073,6.601264,-6.426833,1.042758,4.944042,7.671537,-5.333508,-7.128211,...,1.020387,-3.897649,-7.551146,-4.268405,8.827366,-7.952704,4.574883,-4.473346,8.431083,1


In [8]:
df_latent_test_20dims = pd.read_csv('re-latent_df_test_20dims_PD_HD.csv')
df_latent_test_20dims

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,patient number
0,-6.376636,6.685627,8.486855,-5.005437,0.986963,-2.510397,0.891873,-5.870406,-4.658065,5.980363,...,-5.318729,-6.247756,-2.401874,-9.539203,4.514901,-6.644996,-1.029745,5.043787,-7.681988,0
1,3.641945,-7.661404,7.613710,-1.599616,7.090221,6.726690,-2.069273,-2.745076,2.530561,2.660381,...,8.682987,6.308665,5.456786,8.533322,-3.954587,6.808384,-5.964357,-6.926980,-4.553753,0
2,3.359315,-4.861623,-5.498817,6.374908,1.541136,-6.441969,6.099424,5.147042,-2.580868,-4.977870,...,-3.195096,5.148294,-4.076187,-7.195958,6.914922,-6.155535,4.881458,-7.642689,-8.291637,2
3,-0.756402,0.441134,-6.646461,1.472855,-6.947612,7.541595,4.625729,-6.928267,4.398931,1.071311,...,-5.603864,4.042096,-0.611265,-1.356958,-5.764072,-6.192120,-8.153080,-6.249497,5.730522,1
4,0.938262,-3.290774,6.039089,-9.971478,4.504210,-7.470419,-7.287096,-3.256257,-2.944622,1.461293,...,5.424419,-4.540638,-0.372598,-7.300921,-0.119995,-5.317137,-6.542215,-6.035981,4.083077,1
5,-6.365646,8.686355,-3.822531,-7.310308,-3.612169,-2.193325,-4.453616,-3.896857,2.272247,3.491675,...,-7.702356,1.721456,-6.695879,-4.547217,-7.225779,2.713671,4.677006,1.589107,-5.289326,1


In [9]:
df_train_20dims = df_latent_train_20dims.iloc[:, :-1]
df_train_20dims

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-4.894204,4.146495,-10.627996,4.333094,-0.211089,-6.648519,-4.280519,-3.815852,-3.777365,-1.499009,-0.625212,-7.137741,-5.564987,1.921789,-6.913729,7.227826,-7.480397,-3.841025,-2.643135,-4.290059
1,-2.944945,-4.172675,2.317636,3.224382,2.160191,3.711342,5.053267,-7.703062,8.844686,-1.244160,0.254470,5.201087,-5.925610,4.695848,3.283216,-3.963865,5.261399,-4.708281,2.653075,-5.063646
2,5.588860,-0.694617,2.670450,-8.996168,-3.310686,-7.067754,3.733333,10.910929,2.443390,-4.109570,7.296699,6.122800,6.156366,-1.021480,3.269602,2.010873,1.207475,5.214314,6.464255,-2.690952
3,-7.072881,-4.657294,-6.835857,9.598818,1.465891,-5.823493,-5.777774,-7.312018,-0.630546,-6.401748,-6.745101,3.504016,4.543265,6.947121,8.620768,0.387783,-1.047444,3.723100,5.218976,4.335200
4,-5.568182,9.380668,7.979990,1.130456,5.788285,-3.061255,-3.599283,6.892598,2.539938,2.496101,-4.084094,8.950968,-4.972475,-5.133507,-7.145875,-4.389797,6.115207,-6.742897,5.260499,-5.494330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,3.891827,2.561099,6.970787,-7.018413,-5.253020,3.758243,3.671204,-10.044440,-7.940752,-0.323961,6.115371,-5.004793,1.852236,5.476236,-3.787934,-4.054970,2.748752,6.475208,0.135382,-7.873879
98,-7.752233,-4.231459,8.498736,-5.204750,-0.988339,2.461777,-1.641066,-7.762600,-0.770963,6.920038,-2.429936,-3.921720,-4.659022,-4.321660,2.040658,-4.948811,7.810775,-6.515944,4.377778,4.249447
99,5.738795,3.508221,-3.084219,-2.055181,-6.202135,-2.324691,-3.598113,-4.882000,4.343721,3.288866,-8.758722,-8.157871,-1.671575,-0.765159,-4.126219,5.952621,-7.953374,-6.093229,4.663666,4.219165
100,-1.368667,-3.972085,5.433073,6.601264,-6.426833,1.042758,4.944042,7.671537,-5.333508,-7.128211,-5.145265,1.020387,-3.897649,-7.551146,-4.268405,8.827366,-7.952704,4.574883,-4.473346,8.431083


In [10]:
df_test_20dims = df_latent_test_20dims.iloc[:, :-1]
df_test_20dims

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-6.376636,6.685627,8.486855,-5.005437,0.986963,-2.510397,0.891873,-5.870406,-4.658065,5.980363,6.565961,-5.318729,-6.247756,-2.401874,-9.539203,4.514901,-6.644996,-1.029745,5.043787,-7.681988
1,3.641945,-7.661404,7.613710,-1.599616,7.090221,6.726690,-2.069273,-2.745076,2.530561,2.660381,7.137660,8.682987,6.308665,5.456786,8.533322,-3.954587,6.808384,-5.964357,-6.926980,-4.553753
2,3.359315,-4.861623,-5.498817,6.374908,1.541136,-6.441969,6.099424,5.147042,-2.580868,-4.977870,6.729605,-3.195096,5.148294,-4.076187,-7.195958,6.914922,-6.155535,4.881458,-7.642689,-8.291637
3,-0.756402,0.441134,-6.646461,1.472855,-6.947612,7.541595,4.625729,-6.928267,4.398931,1.071311,-1.451650,-5.603864,4.042096,-0.611265,-1.356958,-5.764072,-6.192120,-8.153080,-6.249497,5.730522
4,0.938262,-3.290774,6.039089,-9.971478,4.504210,-7.470419,-7.287096,-3.256257,-2.944622,1.461293,7.112204,5.424419,-4.540638,-0.372598,-7.300921,-0.119995,-5.317137,-6.542215,-6.035981,4.083077
5,-6.365646,8.686355,-3.822531,-7.310308,-3.612169,-2.193325,-4.453616,-3.896857,2.272247,3.491675,8.014971,-7.702356,1.721456,-6.695879,-4.547217,-7.225779,2.713671,4.677006,1.589107,-5.289326


In [11]:
df_latent_20dims_all = pd.concat([df_latent_train_20dims,df_latent_test_20dims], axis = 0, ignore_index = True)
df_latent_20dims_all

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,patient number
0,-4.894204,4.146495,-10.627996,4.333094,-0.211089,-6.648519,-4.280519,-3.815852,-3.777365,-1.499009,...,-7.137741,-5.564987,1.921789,-6.913729,7.227826,-7.480397,-3.841025,-2.643135,-4.290059,1
1,-2.944945,-4.172675,2.317636,3.224382,2.160191,3.711342,5.053267,-7.703062,8.844686,-1.244160,...,5.201087,-5.925610,4.695848,3.283216,-3.963865,5.261399,-4.708281,2.653075,-5.063646,0
2,5.588860,-0.694617,2.670450,-8.996168,-3.310686,-7.067754,3.733333,10.910929,2.443390,-4.109570,...,6.122800,6.156366,-1.021480,3.269602,2.010873,1.207475,5.214314,6.464255,-2.690952,2
3,-7.072881,-4.657294,-6.835857,9.598818,1.465891,-5.823493,-5.777774,-7.312018,-0.630546,-6.401748,...,3.504016,4.543265,6.947121,8.620768,0.387783,-1.047444,3.723100,5.218976,4.335200,1
4,-5.568182,9.380668,7.979990,1.130456,5.788285,-3.061255,-3.599283,6.892598,2.539938,2.496101,...,8.950968,-4.972475,-5.133507,-7.145875,-4.389797,6.115207,-6.742897,5.260499,-5.494330,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,3.641945,-7.661404,7.613710,-1.599616,7.090221,6.726690,-2.069273,-2.745076,2.530561,2.660381,...,8.682987,6.308665,5.456786,8.533322,-3.954587,6.808384,-5.964357,-6.926980,-4.553753,0
104,3.359315,-4.861623,-5.498817,6.374908,1.541136,-6.441969,6.099424,5.147042,-2.580868,-4.977870,...,-3.195096,5.148294,-4.076187,-7.195958,6.914922,-6.155535,4.881458,-7.642689,-8.291637,2
105,-0.756402,0.441134,-6.646461,1.472855,-6.947612,7.541595,4.625729,-6.928267,4.398931,1.071311,...,-5.603864,4.042096,-0.611265,-1.356958,-5.764072,-6.192120,-8.153080,-6.249497,5.730522,1
106,0.938262,-3.290774,6.039089,-9.971478,4.504210,-7.470419,-7.287096,-3.256257,-2.944622,1.461293,...,5.424419,-4.540638,-0.372598,-7.300921,-0.119995,-5.317137,-6.542215,-6.035981,4.083077,1


In [12]:
df_latent_20dims = df_latent_20dims_all.iloc[:, :-1]
df_latent_20dims

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-4.894204,4.146495,-10.627996,4.333094,-0.211089,-6.648519,-4.280519,-3.815852,-3.777365,-1.499009,-0.625212,-7.137741,-5.564987,1.921789,-6.913729,7.227826,-7.480397,-3.841025,-2.643135,-4.290059
1,-2.944945,-4.172675,2.317636,3.224382,2.160191,3.711342,5.053267,-7.703062,8.844686,-1.244160,0.254470,5.201087,-5.925610,4.695848,3.283216,-3.963865,5.261399,-4.708281,2.653075,-5.063646
2,5.588860,-0.694617,2.670450,-8.996168,-3.310686,-7.067754,3.733333,10.910929,2.443390,-4.109570,7.296699,6.122800,6.156366,-1.021480,3.269602,2.010873,1.207475,5.214314,6.464255,-2.690952
3,-7.072881,-4.657294,-6.835857,9.598818,1.465891,-5.823493,-5.777774,-7.312018,-0.630546,-6.401748,-6.745101,3.504016,4.543265,6.947121,8.620768,0.387783,-1.047444,3.723100,5.218976,4.335200
4,-5.568182,9.380668,7.979990,1.130456,5.788285,-3.061255,-3.599283,6.892598,2.539938,2.496101,-4.084094,8.950968,-4.972475,-5.133507,-7.145875,-4.389797,6.115207,-6.742897,5.260499,-5.494330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,3.641945,-7.661404,7.613710,-1.599616,7.090221,6.726690,-2.069273,-2.745076,2.530561,2.660381,7.137660,8.682987,6.308665,5.456786,8.533322,-3.954587,6.808384,-5.964357,-6.926980,-4.553753
104,3.359315,-4.861623,-5.498817,6.374908,1.541136,-6.441969,6.099424,5.147042,-2.580868,-4.977870,6.729605,-3.195096,5.148294,-4.076187,-7.195958,6.914922,-6.155535,4.881458,-7.642689,-8.291637
105,-0.756402,0.441134,-6.646461,1.472855,-6.947612,7.541595,4.625729,-6.928267,4.398931,1.071311,-1.451650,-5.603864,4.042096,-0.611265,-1.356958,-5.764072,-6.192120,-8.153080,-6.249497,5.730522
106,0.938262,-3.290774,6.039089,-9.971478,4.504210,-7.470419,-7.287096,-3.256257,-2.944622,1.461293,7.112204,5.424419,-4.540638,-0.372598,-7.300921,-0.119995,-5.317137,-6.542215,-6.035981,4.083077


In [13]:
model20dims = load_model("model20_11_20dim.h5")
model20dims.summary()

2025-06-23 12:56:28.370787: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-23 12:56:28.372141: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2025-06-23 12:56:28.514131: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-06-23 12:56:28.514840: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2025-06-23 12:56

Model: "sequential_223"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_914 (Dense)            (None, 100)               2186200   
_________________________________________________________________
batch_normalization_693 (Bat (None, 100)               400       
_________________________________________________________________
dense_915 (Dense)            (None, 100)               10100     
_________________________________________________________________
batch_normalization_694 (Bat (None, 100)               400       
_________________________________________________________________
dense_916 (Dense)            (None, 20)                2020      
_________________________________________________________________
batch_normalization_695 (Bat (None, 20)                80        
_________________________________________________________________
dense_917 (Dense)            (None, 100)            

In [14]:
# Extract the shape of the output from the 'dense_916' layer
intermediate_layer_output_shape = model20dims.get_layer('dense_916').output_shape[1:]

# Define a new input tensor with the same shape as the output of 'dense_53'
new_input = Input(shape=intermediate_layer_output_shape)

# Apply the layers after 'dense_916' to this new input tensor
x = model20dims.get_layer('batch_normalization_695')(new_input)
x = model20dims.get_layer('dense_917')(x)
x = model20dims.get_layer('batch_normalization_696')(x)
x = model20dims.get_layer('dense_918')(x)
x = model20dims.get_layer('batch_normalization_697')(x)
output = model20dims.get_layer('dense_919')(x)

# Create the new model from 'dense_916' to 'dense_919'
decoder_model_20dims = Model(inputs=new_input, outputs=output)

# Display the summary of the new model
decoder_model_20dims.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
batch_normalization_695 (Bat (None, 20)                80        
_________________________________________________________________
dense_917 (Dense)            (None, 100)               2100      
_________________________________________________________________
batch_normalization_696 (Bat (None, 100)               400       
_________________________________________________________________
dense_918 (Dense)            (None, 100)               10100     
_________________________________________________________________
batch_normalization_697 (Bat (None, 100)               400       
_________________________________________________________________
dense_919 (Dense)            (None, 21861)             220796

In [15]:
# decoder data extract
decoder_train_np = decoder_model_20dims(df_train_20dims.values).numpy()
decoder_train_np.shape

2025-06-23 12:56:34.445945: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


(102, 21861)

In [16]:
# decoder data extract
decoder_test_np = decoder_model_20dims(df_test_20dims.values).numpy()
decoder_test_np.shape

(6, 21861)

In [17]:
decoder_data = np.concatenate([decoder_train_np, decoder_test_np], axis = 0)
decoder_data.shape

(108, 21861)

In [18]:
# calculate gradient means from latent layer to decoder layer at a specific column in latent space
def latent_gradient_mean_each_gene(decoder_model, x, idx_genes, latent_dims):
    x_tensor = tf.convert_to_tensor(x, dtype=tf.float32)
    with tf.GradientTape() as t:
        t.watch(x_tensor)
        loss = decoder_model(x_tensor)
        loss_idx_genes =loss[:,idx_genes:idx_genes+1]
        G = t.gradient(loss_idx_genes, x_tensor).numpy() # shape(102, 20)
    return np.mean(G[:, latent_dims-1])

In [19]:
gradient_means_16 = []
for i in range(21861):
    grad_mean_16 = latent_gradient_mean_each_gene(decoder_model_20dims, df_latent_20dims, i, 16)
    gradient_means_16.append(grad_mean_16)
    print(i+1,' genes')
    print(grad_mean_16)

1  genes
0.0040548835
2  genes
-0.004755607
3  genes
0.0035995182
4  genes
0.0069907005
5  genes
0.00544991
6  genes
-0.007194649
7  genes
0.0037765827
8  genes
0.0043521984
9  genes
0.002801792
10  genes
0.0027900082
11  genes
0.004258369
12  genes
0.0013536275
13  genes
-0.007841594
14  genes
0.0025386896
15  genes
-0.0029482518
16  genes
-0.0007524945
17  genes
0.001880861
18  genes
0.008008527
19  genes
0.0022565792
20  genes
-0.011263235
21  genes
0.0055399183
22  genes
0.008835273
23  genes
-0.0025855508
24  genes
-0.0029163326
25  genes
0.004190202
26  genes
0.0022451892
27  genes
0.0044635795
28  genes
0.0020570096
29  genes
0.0027857837
30  genes
0.0021999122
31  genes
-0.0021927434
32  genes
0.0036321525
33  genes
0.004858855
34  genes
-0.0015113063
35  genes
0.0017431616
36  genes
0.0029510912
37  genes
0.0035880657
38  genes
0.0046873745
39  genes
-0.0019859476
40  genes
0.005009951
41  genes
0.0069162482
42  genes
-0.0023783816
43  genes
0.0060546133
44  genes
0.0007262461

In [20]:
gradient_means_16_np = np.array(gradient_means_16)
gradient_means_16_np.shape

(21861,)

In [21]:
data = pd.read_csv('tpm_data.csv', header = 0)
data

,Unnamed: 0,ENSG,hgnc_symbol,gene_length,C_0002,C_0003,C_0004,C_0005,C_0006,C_0008,...,H_0740,H_0750,H_0513,H_0601,H_0656,H_0689,H_0709,H_0723,H_1104,H_1105
0,1,ENSG00000000003,TSPAN6,12883,2.703162,4.679815,3.827170,3.940771,3.956139,4.749555,...,4.799810,8.733498,3.666497,4.563176,2.492904,2.949728,3.314066,6.985665,3.947979,3.784485
1,2,ENSG00000000005,TNMD,14949,0.005418,0.029438,0.088121,0.036995,0.053272,0.108531,...,0.024476,0.086370,0.083015,0.078651,0.050749,0.014444,0.028001,0.138079,0.031214,0.064583
2,3,ENSG00000000419,DPM1,24273,2.625862,4.387499,4.450224,5.390720,3.707350,5.356800,...,4.100151,6.470481,5.783705,6.878268,3.167153,1.530000,6.380522,8.912018,6.420788,9.267549
3,4,ENSG00000000457,SCYL3,44636,0.896318,1.025353,1.568379,1.315815,1.617599,1.350063,...,0.610696,0.892573,1.098204,1.589228,0.708180,0.416006,0.628301,1.156094,1.317197,2.011547
4,5,ENSG00000000460,C1orf112,192073,0.033732,0.123724,0.065645,0.074862,0.063574,0.114637,...,0.030003,0.052817,0.036344,0.091820,0.025015,0.042717,0.034868,0.060181,0.072882,0.070371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21858,26560,ENSG00000273372,SFTPD-AS1,5000,0.680299,0.616103,0.715117,1.061834,1.061811,1.437003,...,0.311009,0.202893,0.201662,1.018982,0.050577,0.647750,0.000000,0.247696,0.559947,0.096546
21859,26565,ENSG00000273396,LINC01396,6639,0.036596,0.066286,0.226767,0.049981,0.119952,0.139644,...,0.013778,0.027783,0.000000,0.000000,0.038090,0.000000,0.000000,0.000000,0.000000,0.000000
21860,26570,ENSG00000273409,LINC02712,66004,0.019632,0.040004,0.068428,0.016758,0.036196,0.017558,...,0.005544,0.027945,0.054055,0.065315,0.015325,0.009814,0.000000,0.012509,0.028278,0.058509
21861,26571,ENSG00000273415,LINC02725,87913,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [22]:
data_del2=data.drop(data.index[[13609,19081]])
data_del2

,Unnamed: 0,ENSG,hgnc_symbol,gene_length,C_0002,C_0003,C_0004,C_0005,C_0006,C_0008,...,H_0740,H_0750,H_0513,H_0601,H_0656,H_0689,H_0709,H_0723,H_1104,H_1105
0,1,ENSG00000000003,TSPAN6,12883,2.703162,4.679815,3.827170,3.940771,3.956139,4.749555,...,4.799810,8.733498,3.666497,4.563176,2.492904,2.949728,3.314066,6.985665,3.947979,3.784485
1,2,ENSG00000000005,TNMD,14949,0.005418,0.029438,0.088121,0.036995,0.053272,0.108531,...,0.024476,0.086370,0.083015,0.078651,0.050749,0.014444,0.028001,0.138079,0.031214,0.064583
2,3,ENSG00000000419,DPM1,24273,2.625862,4.387499,4.450224,5.390720,3.707350,5.356800,...,4.100151,6.470481,5.783705,6.878268,3.167153,1.530000,6.380522,8.912018,6.420788,9.267549
3,4,ENSG00000000457,SCYL3,44636,0.896318,1.025353,1.568379,1.315815,1.617599,1.350063,...,0.610696,0.892573,1.098204,1.589228,0.708180,0.416006,0.628301,1.156094,1.317197,2.011547
4,5,ENSG00000000460,C1orf112,192073,0.033732,0.123724,0.065645,0.074862,0.063574,0.114637,...,0.030003,0.052817,0.036344,0.091820,0.025015,0.042717,0.034868,0.060181,0.072882,0.070371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21858,26560,ENSG00000273372,SFTPD-AS1,5000,0.680299,0.616103,0.715117,1.061834,1.061811,1.437003,...,0.311009,0.202893,0.201662,1.018982,0.050577,0.647750,0.000000,0.247696,0.559947,0.096546
21859,26565,ENSG00000273396,LINC01396,6639,0.036596,0.066286,0.226767,0.049981,0.119952,0.139644,...,0.013778,0.027783,0.000000,0.000000,0.038090,0.000000,0.000000,0.000000,0.000000,0.000000
21860,26570,ENSG00000273409,LINC02712,66004,0.019632,0.040004,0.068428,0.016758,0.036196,0.017558,...,0.005544,0.027945,0.054055,0.065315,0.015325,0.009814,0.000000,0.012509,0.028278,0.058509
21861,26571,ENSG00000273415,LINC02725,87913,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [23]:
genes = data_del2['hgnc_symbol'].values
genes

array(['TSPAN6', 'TNMD', 'DPM1', ..., 'LINC02712', 'LINC02725', 'APP-DT'],
      dtype=object)

In [25]:
df_genes_grad_16 = pd.DataFrame({'genes':genes, 'gradients':gradient_means_16_np})
df_genes_grad_16

,genes,gradients
0,TSPAN6,0.004055
1,TNMD,-0.004756
2,DPM1,0.003600
3,SCYL3,0.006991
4,C1orf112,0.005450
...,...,...
21856,SFTPD-AS1,-0.005916
21857,LINC01396,-0.004225
21858,LINC02712,0.008623
21859,LINC02725,0.023172


In [26]:
# Calculate the 95th and 5th percentile thresholds for the gradient values
upper_threshold = df_genes_grad_16['gradients'].quantile(0.95)
lower_threshold = df_genes_grad_16['gradients'].quantile(0.05)

# Extract gene names with gradient values in the top 5% and bottom 5%
top_5_percent_genes_16 = df_genes_grad_16[df_genes_grad_16['gradients'] >= upper_threshold]['genes']
bottom_5_percent_genes_16 = df_genes_grad_16[df_genes_grad_16['gradients'] <= lower_threshold]['genes']

# Display the results
print("Top 5% Gene Names:")
print(top_5_percent_genes_16.tolist())

print("Bottom 5% Gene Names:")
print(bottom_5_percent_genes_16.tolist())

Top 5% Gene Names:
['THSD7A', 'TTC22', 'LGALS14', 'PROM1', 'PLEKHG6', 'SEC62', 'SLC6A13', 'FMO1', 'UTP18', 'DCN', 'ALOX5', 'CD6', 'SLC30A9', 'RB1CC1', 'HSD17B6', 'MIPEP', 'RABEP1', 'HMGB3', 'STAP1', 'BEST2', 'BARX2', 'EPHA3', 'MAGEC2', 'KCNQ1', 'NDC1', 'BTBD1', 'CD84', 'ACSM2B', 'IARS2', 'STON1-GTF2A1L', 'ADGRF5', 'RPL31', 'TRIB2', 'ST6GAL1', 'SRI', 'ACAT1', 'ARHGAP15', 'SPAG6', 'TIGAR', 'CDH17', 'XRCC5', 'SP140', 'AFP', 'COL4A4', 'PTPRC', 'FYB1', 'COBLL1', 'LYRM2', 'EPYC', 'PPIE', 'IPO11', 'CETP', 'F11', 'SNX5', 'MRPS33', 'LYZ', 'FETUB', 'KIF4A', 'FH', 'TBL1Y', 'WDR76', 'CLSPN', 'CDC45', 'BLNK', 'MYO3A', 'TPSD1', 'CENPM', 'DMC1', 'HSCB', 'PNPLA3', 'CSF2RB', 'IL2RB', 'BRMS1L', 'PROCR', 'SMARCA1', 'MTMR8', 'OPN1LW', 'CYB5B', 'PRSS54', 'CLUAP1', 'SLC6A2', 'ATP8B4', 'EIF3J', 'DECR1', 'EMC2', 'CCN4', 'ADAM2', 'CGB2', 'DKKL1', 'KIR3DX1', 'LILRA1', 'URI1', 'CRX', 'SIGLEC5', 'CABP5', 'LILRB5', 'SLC5A5', 'TFPI2', 'BET1', 'PTN', 'TFEC', 'MET', 'WNT2', 'CHCHD2', 'PTPRZ1', 'RPA3', 'DOCK8', 'CXCL1

In [28]:
top_5_percent_genes_16

80            THSD7A
139            TTC22
148          LGALS14
164            PROM1
213          PLEKHG6
            ...     
21817      LRRC8D-DT
21819    RBAK-RBAKDN
21836      LINC01990
21844           H3-7
21859      LINC02725
Name: genes, Length: 1094, dtype: object

In [29]:
genes_from_16_20dims = pd.concat([top_5_percent_genes_16, bottom_5_percent_genes_16], axis = 0)
genes_from_16_20dims

80           THSD7A
139           TTC22
148         LGALS14
164           PROM1
213         PLEKHG6
            ...    
21795    TIMMDC1-DT
21804    SLC30A6-DT
21806     ADIRF-AS1
21834         LYPD4
21839     LINC02604
Name: genes, Length: 2188, dtype: object

In [30]:
genes_from_16_20dims.to_csv('genes_for_metascape_from_feature16_20dims_gradient_top_bottom_5%_all.csv', index = False)